# Stock Price Prediction with LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

## 1. Load and Preprocess Data

In [ ]:
df = pd.read_csv('synthetic_stock_data.csv')
data = df['Price'].values.reshape(-1, 1)

# Scale the data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

## 2. Create Training Sequences
LSTMs need data in sequences. We'll use the last 60 data points (timesteps) to predict the next one.

In [ ]:
def create_sequences(data, timesteps):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:(i + timesteps), 0])
        y.append(data[i + timesteps, 0])
    return np.array(X), np.array(y)

timesteps = 60
X, y = create_sequences(scaled_data, timesteps)

# Reshape X to be [samples, timesteps, features] for the LSTM layer
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

## 3. Build and Train the LSTM Model

In [ ]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)),
    LSTM(units=50),
    Dense(units=25),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
# Train the model
model.fit(X, y, batch_size=32, epochs=10)

## 4. Save the Model and Scaler

In [ ]:
import joblib
# Ensure models directory exists
if not os.path.exists('models'):
    os.makedirs('models')

# Save the trained model
model.save('models/lstm_stock_predictor.keras')

# Save the scaler object, which is needed to reverse the prediction scaling
joblib.dump(scaler, 'models/scaler.pkl')

print("Model and scaler saved successfully.")